In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [51]:
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir

df = pd.read_csv(latestdir+'/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv',encoding='latin1',delimiter = ';')
df['Prøvetagningsdato'] = pd.to_datetime(df['Prøvetagningsdato'])

kortdf  = pd.read_csv(latestdir+'/Regionalt_DB/13_regionale_kort.csv',encoding='latin1',delimiter = ';')
kortdf

,Region,Bekræftede tilfælde,Incidens,Bekræftede tilfælde de senest 7 dage,Incidens de seneste 7 dage,Testede,Test incidens,Testede de seneste 7 dage,Test incidens de seneste 7 dage,Positivprocent de seneste 7 dage,antal_bor
0,Hovedstaden,380466,20433,45170,2426,1722963,92531,288509,15494,"15,66",1862046
1,Midtjylland,144153,10762,21640,1616,1188631,88739,180298,13460,"12,00",1339467
2,Nordjylland,60170,10178,9437,1596,538273,91049,72188,12211,"13,07",591192
3,Sjælland,111256,13218,15625,1856,743982,88388,121950,14488,"12,81",841723
4,Syddanmark,120667,9836,18370,1497,1092441,89053,183793,14982,"9,99",1226727


In [55]:
allRegions = kortdf.Region.values
borgerCount = kortdf.antal_bor.values


In [56]:
# df.groupby('Køn').sum()
# allRegions = df.Region.unique()

groupdf = df.groupby(['Region','Prøvetagningsdato']).sum()
# groupdf[groupdf.index == 'Hovedstaden']
# groupdf.index.names
curdf = groupdf.loc[allRegions[1],:,:]
curDates = curdf.index.get_level_values(1)
curCase = curdf['Bekræftede tilfælde']
curAdm = curdf['Indlæggelser']
curDea = curdf['Døde']

fig,ax1 = plt.subplots()

ax1.plot(curDates,curCase)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:

firstDate = np.datetime64('2021-10-01')
lastDate = np.datetime64('2022-02-01')

firstDateShort = np.datetime64('2021-10')
lastDateShort = np.datetime64('2022-02')
curXticks = np.arange(firstDateShort,lastDateShort+np.timedelta64(1,'M'))

meanWidth= 7

In [218]:
# Compare cases

fig,allAxes = plt.subplots(2,3,figsize=(16,6),tight_layout=True)

for k in range(1,6):
    curRegion = allRegions[k-1]
    # curBorgerCount = borgerCount[k-1]
    curdf = groupdf.loc[curRegion,:,:]
    curDates = curdf.index.get_level_values(1)
    curCase = curdf['Bekræftede tilfælde']
    # curAdm = curdf['Indlæggelser']
    # curDea = curdf['Døde']

    ax = allAxes.flatten()[k]
    # ax.plot(curDates,curCase)
    # ax.plot(curDates[:-1],curCase[:-1])
    ax.plot(curDates[:-1],curCase[:-1],'k.:',markersize=4,linewidth=0.5,label='Data')
    ax.plot(rnTime(curDates[:-1],meanWidth),rnMean(curCase[:-1],meanWidth),'k',label=f'{meanWidth} dages gennemsnit')

    ax.set_xlim(firstDateShort,lastDateShort+np.timedelta64(12,'h'))
    ax.set_xticks(curXticks)
    ax.set_ylim(bottom=0)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False) 
    ax.set_title(curRegion)
    ax.grid()

ax1 = allAxes.flatten()[0]
ax1.axis('off')
txt1 = 'Antal daglige\ntilfælde'
txt2 = 'Rasmus Kristoffer Pedersen\n PandemiX Center, Roskilde Universitet\n\n\nData fra SSI overvågningsdata, dashboard-fil\n\nFilnavn:\n/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv'
ax1.text(0.5, 0.55,txt1,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=20)
ax1.text(0.5, 0,txt2,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=7)

allAxes.flatten()[1].legend(loc='upper left',fontsize=10)

if saveFigures:
     plt.savefig(path_figs+'/Regionalt/Cases')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [217]:
# Compare admissions
fig,allAxes = plt.subplots(2,3,figsize=(16,6),tight_layout=True)

for k in range(1,6):
    curRegion = allRegions[k-1]
    # curBorgerCount = borgerCount[k-1]
    curdf = groupdf.loc[curRegion,:,:]
    curDates = curdf.index.get_level_values(1)
    # curCase = curdf['Bekræftede tilfælde']
    curAdm = curdf['Indlæggelser']
    # curDea = curdf['Døde']

    ax = allAxes.flatten()[k]
    # ax.plot(curDates,curAdm)
    # ax.plot(curDates[:-1],curAdm[:-1])
    ax.plot(curDates[:-1],curAdm[:-1],'k.:',markersize=4,linewidth=0.5,label='Data')
    ax.plot(rnTime(curDates[:-1],meanWidth),rnMean(curAdm[:-1],meanWidth),'k',label=f'{meanWidth} dages gennemsnit')

    ax.set_xlim(firstDateShort,lastDateShort+np.timedelta64(12,'h'))
    ax.set_xticks(curXticks)
    ax.set_ylim(bottom=0)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False) 
    ax.set_title(curRegion)
    ax.grid()

ax1 = allAxes.flatten()[0]
ax1.axis('off')
txt1 = 'Antal daglige\nnyindlæggelser'
txt2 = 'Rasmus Kristoffer Pedersen\n PandemiX Center, Roskilde Universitet\n\n\nData fra SSI overvågningsdata, dashboard-fil\n\nFilnavn:\n/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv'
# txt2 = 'Data fra SSI overvågningsdata, dashboard-fil\n\nFilnavn:\n/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv\n\nRasmus Kristoffer Pedersen\n PandemiX Center, Roskilde Universitet'
ax1.text(0.5, 0.55,txt1,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=20)
ax1.text(0.5, 0,txt2,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=7)

allAxes.flatten()[1].legend(loc='upper left',fontsize=10)

if saveFigures:
     plt.savefig(path_figs+'/Regionalt/Nyindlagte')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [222]:
# Compare cases, normalized by antal_bor


fig,allAxes = plt.subplots(2,3,sharey=True,figsize=(16,6),tight_layout=True)

# allMaxs = []

for k in range(1,6):
    curRegion = allRegions[k-1]
    curBorgerCount = borgerCount[k-1]
    curdf = groupdf.loc[curRegion,:,:]
    curDates = curdf.index.get_level_values(1)
    curCase = curdf['Bekræftede tilfælde']

    ax = allAxes.flatten()[k]

    # ax.plot(curDates[:-1],curCase[:-1]/curBorgerCount,'k.:',markersize=4,linewidth=0.5)
    # ax.plot(rnTime(curDates[:-1],meanWidth),rnMean(curCase[:-1]/curBorgerCount,meanWidth),'k')
    ax.plot(curDates[:-1],100*curCase[:-1]/curBorgerCount,'k.:',markersize=4,linewidth=0.5)
    ax.plot(rnTime(curDates[:-1],meanWidth),rnMean(100*curCase[:-1]/curBorgerCount,meanWidth),'k')

    ax.set_xlim(firstDateShort,lastDateShort+np.timedelta64(12,'h'))
    ax.set_xticks(curXticks)
    ax.set_ylim(bottom=0)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False) 
    ax.set_title(curRegion)
    ax.grid()
    ax.yaxis.set_tick_params(labelleft=True)

ax1 = allAxes.flatten()[0]
ax1.axis('off')
txt1 = 'Tilfælde \n[% af befolkning]'
txt2 = 'Rasmus Kristoffer Pedersen\n PandemiX Center, Roskilde Universitet\n\n\nData fra SSI overvågningsdata, dashboard-fil\n\nFilnavn:\n/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv\nsamt indbyggertal fra: /Regionalt_DB/13_regionale_kort.csv.csv'
ax1.text(0.5, 0.55,txt1,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=20)
ax1.text(0.5, 0,txt2,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=7)

     
if saveFigures:
     plt.savefig(path_figs+'/Regionalt/CasesPercent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [154]:
# # len(curdf['Bekræftede tilfælde'])
# plt.figure()
# plt.plot(curdf['Bekræftede tilfælde'].values[::7])
curDates[0]

numpy.datetime64('2020-03-02T00:00:00.000000000')

In [225]:
# Compare cases, normalized by antal_bor


fig,allAxes = plt.subplots(2,3,sharey=True,figsize=(16,6),tight_layout=True)

# allMaxs = []

for k in range(1,6):
    curRegion = allRegions[k-1]
    curBorgerCount = borgerCount[k-1]
    curdf = groupdf.loc[curRegion,:,:]
    curDates = curdf.index.get_level_values(1).values[:-1]
    curCase = curdf['Bekræftede tilfælde'].values[:-1]

    ax = allAxes.flatten()[k]

    meanDates = rnTime(curDates,meanWidth)
    meanCase = meanWidth*100*rnMean(curCase,meanWidth)/curBorgerCount
    # ax.step(meanDates,meanCase,'k',where='mid')
    # ax.step(meanDates[5::7],meanCase[5::7],'k')
    # ax.step(meanDates[::7],meanCase[::7],'k',where='mid')
    ax.step(meanDates[1::7],meanCase[1::7],'k',where='pre')
    ax.fill_between(meanDates[1::7],meanCase[1::7],step='pre',color='k',alpha=0.5)

    ax.set_xlim(firstDateShort,lastDateShort+np.timedelta64(12,'h'))
    ax.set_xticks(curXticks)
    ax.set_ylim(bottom=0)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False) 
    ax.set_title(curRegion)
    ax.grid()
    ax.yaxis.set_tick_params(labelleft=True)

ax1 = allAxes.flatten()[0]
ax1.axis('off')
# # ax1.set_title('Tilfælde \n[% af befolkning]')
# ax1.text(0.5, 0.75,'Sum af ugentlige tilfælde \n[% af befolkning]',
#      horizontalalignment='center',
#      verticalalignment='center',
#      transform = ax1.transAxes,
#      fontsize=20)

txt1 = 'Sum af ugentlige tilfælde \n[% af befolkning]'
txt2 = 'Rasmus Kristoffer Pedersen\n PandemiX Center, Roskilde Universitet\n\n\nData fra SSI overvågningsdata, dashboard-fil\n\nFilnavn:\n/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv\nsamt indbyggertal fra: /Regionalt_DB/13_regionale_kort.csv.csv'
ax1.text(0.5, 0.55,txt1,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=20)
ax1.text(0.5, 0,txt2,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=7)
     
if saveFigures:
     plt.savefig(path_figs+'/Regionalt/CasesPercentWeek')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

120666

In [316]:
# Compare cases, normalized by antal_bor


fig,allAxes = plt.subplots(2,3,figsize=(16,6),tight_layout=True)

# allMaxs = []

for k in range(1,6):
    curRegion = allRegions[k-1]
    curBorgerCount = borgerCount[k-1]
    curdf = groupdf.loc[curRegion,:,:]
    curDates = curdf.index.get_level_values(1).values[:-1]
    curCase = curdf['Bekræftede tilfælde'].values[:-1]
    curCumSum = np.cumsum(curCase)

    ax = allAxes.flatten()[k]

    ax.plot(curDates,100*curCumSum/curBorgerCount,'k')

    # Calculate difference
    startIndex = np.where(curDates == firstDate)[0][0]

    latestSum = 100 * curCumSum[-1]/curBorgerCount
    startSum =  100 * curCumSum[startIndex]/curBorgerCount
    midSum = (latestSum+startSum)/2
    diffSum = latestSum-startSum

    annoDate = lastDate-np.timedelta64(14,'D')
    annoDate1 = annoDate - np.timedelta64(3,'D')
    annoDate2 = annoDate + np.timedelta64(3,'D')
    annoColor = 'b'
    # ax.plot([annoDate,annoDate],[startSum,latestSum],'m_-',markersize=20,linewidth=3)
    ax.plot([annoDate,annoDate],[startSum,latestSum],color=annoColor,linewidth=2)
    ax.plot([annoDate1,annoDate2],[startSum,startSum],color=annoColor,linewidth=2)
    ax.plot([annoDate1,annoDate2],[latestSum,latestSum],color=annoColor,linewidth=2)
    ax.text(annoDate2,midSum,f'{diffSum:2.0f}%',color=annoColor,fontsize=11,fontvariant='small-caps')#fontfamily='serif')



    # Make figure nice
    ax.set_xlim(firstDateShort,lastDateShort+np.timedelta64(12,'h'))
    ax.set_xticks(curXticks)
    ax.set_yticks(np.arange(0,50,5))
    ax.set_ylim(bottom=0)
    ax.set_ylim(top=latestSum*1.2)
    ax.set_ylim(top=np.ceil(latestSum*1.2/5)*5)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False) 
    ax.set_title(curRegion)
    ax.grid()
    ax.yaxis.set_tick_params(labelleft=True)

ax1 = allAxes.flatten()[0]
ax1.axis('off')

txt1 = 'Kumuleret sum af tilfælde \n[% af befolkning]'
txt2 = 'Rasmus Kristoffer Pedersen\n PandemiX Center, Roskilde Universitet\n\n\nData fra SSI overvågningsdata, dashboard-fil\n\nFilnavn:\n/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv\nsamt indbyggertal fra: /Regionalt_DB/13_regionale_kort.csv.csv'
ax1.text(0.5, 0.55,txt1,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=20)
ax1.text(0.5, 0,txt2,
     horizontalalignment='center',
     verticalalignment='bottom',
     transform = ax1.transAxes,
     fontsize=7)
     
if saveFigures:
     plt.savefig(path_figs+'/Regionalt/CasesCumulative')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [275]:
# curCase[-10:]
# curdf
# df[df.Region == curRegion]
# df
latestSum

0.09836418371813778

In [123]:
print(kortdf['Bekræftede tilfælde de senest 7 dage']/kortdf['antal_bor'])
print(kortdf['Bekræftede tilfælde']/kortdf['antal_bor'])
kortdf

0    0.024258
1    0.016156
2    0.015963
3    0.018563
4    0.014975
dtype: float64
0    0.204327
1    0.107620
2    0.101777
3    0.132176
4    0.098365
dtype: float64


,Region,Bekræftede tilfælde,Incidens,Bekræftede tilfælde de senest 7 dage,Incidens de seneste 7 dage,Testede,Test incidens,Testede de seneste 7 dage,Test incidens de seneste 7 dage,Positivprocent de seneste 7 dage,antal_bor
0,Hovedstaden,380466,20433,45170,2426,1722963,92531,288509,15494,"15,66",1862046
1,Midtjylland,144153,10762,21640,1616,1188631,88739,180298,13460,"12,00",1339467
2,Nordjylland,60170,10178,9437,1596,538273,91049,72188,12211,"13,07",591192
3,Sjælland,111256,13218,15625,1856,743982,88388,121950,14488,"12,81",841723
4,Syddanmark,120667,9836,18370,1497,1092441,89053,183793,14982,"9,99",1226727


In [102]:
45000/1862046

0.024166964725898285